# 方策反復法

方策反復法では、価値更新を紙ではなくコードで追い、再帰の意味を体感します。

前提: 期待値、再帰的な定義、逐次意思決定の基本が前提です。

到達目標: ベルマン方程式の見方を、価値更新のコードとセットで理解し、手法差を説明できる状態にします。

ここで扱う中心語は 「状態」、「行動」、「報酬」、「価値関数」、「方策」、「方策反復法」 です。用語を先に暗記するのではなく、コード実行の結果と結びつけて理解します。

このノートは 強化学習 分野の初学者向けに、説明とコードを交互に読み進める設計です。最初から完璧に理解する必要はありません。大切なのは、各コードの目的を一文で言えることと、出力が変わる理由を自分で確かめることです。

## 1. 報酬の割引和を計算する

まず、将来報酬を現在価値へ割り引く計算を実装します。価値関数の直感はこの計算から始まります。

ここで注目してほしいのは、処理の入出力がどの変数で接続されるかです。コードを読むときは『この行は何を決める行か』を短くメモしながら進むと理解が安定します。

In [ ]:
rewards = [0, 0, 1, 2]
gamma = 0.9
g = 0.0
for r in reversed(rewards):
    g = r + gamma * g
print('return =', round(g, 6))

割引率 `gamma` を下げると短期志向になり、上げると長期志向になります。設計意図に合わせて選びます。

たとえば 状態 を別の値に変更すると、出力がどう変わるかを確認してください。次に、同じコードで境界条件を試し、なぜその結果になるのかを言葉で説明します。

この確認を繰り返すと、理論と実装の対応が頭の中で一本化されます。

## 2. ベルマン更新を1回行う

次に、価値更新を1ステップだけ計算します。1回更新でも、再帰構造の意味は十分に見えてきます。

ここで注目してほしいのは、処理の入出力がどの変数で接続されるかです。コードを読むときは『この行は何を決める行か』を短くメモしながら進むと理解が安定します。

In [ ]:
v_next = {'s0': 0.4, 's1': 0.8}
reward = {'left': 0.2, 'right': 1.0}
trans = {'left': 's0', 'right': 's1'}
v_s = max(reward[a] + gamma * v_next[trans[a]] for a in ['left', 'right'])
print('updated V(s)=', round(v_s, 6))

ベルマン更新は『今の価値』を『次状態の価値』で再定義する操作です。この再帰が強化学習の中心です。

たとえば 状態 を別の値に変更すると、出力がどう変わるかを確認してください。次に、同じコードで境界条件を試し、なぜその結果になるのかを言葉で説明します。

この確認を繰り返すと、理論と実装の対応が頭の中で一本化されます。

## 式をコードと接続する

ここで、本文中で使っている式を実装視点で再整理します。式を暗記するのではなく、どの変数がどのコード行に対応するかを確認してください。

1. G_t = sum_{k>=0} gamma^k R_{t+k+1}
2. Q <- Q + alpha * TD_error

この一覧は『正解の丸暗記』ではなく、デバッグ時の観点表です。実装が壊れたとき、どの式のどの項が怪しいかを逆算できるように使います。

## 3. Q値更新を比較する

ここで Q学習の更新式をコードに写し、数値の動きを確認します。式を読むだけでは掴みにくい感覚を得る段階です。

ここで注目してほしいのは、処理の入出力がどの変数で接続されるかです。コードを読むときは『この行は何を決める行か』を短くメモしながら進むと理解が安定します。

In [ ]:
Q = {('s0','left'): 0.3, ('s0','right'): 0.1, ('s1','left'): 0.5, ('s1','right'): 0.7}
alpha = 0.2
r, s, a, s_next = 1.0, 's0', 'right', 's1'
td_target = r + gamma * max(Q[(s_next,'left')], Q[(s_next,'right')])
Q[(s,a)] += alpha * (td_target - Q[(s,a)])
print('Q(s0,right)=', round(Q[(s,a)], 6))

更新後の値が過去の値とどれだけ違うかは、学習率と TD 誤差で決まります。ここが調整ポイントです。

たとえば 状態 を別の値に変更すると、出力がどう変わるかを確認してください。次に、同じコードで境界条件を試し、なぜその結果になるのかを言葉で説明します。

この確認を繰り返すと、理論と実装の対応が頭の中で一本化されます。

## 4. 探索と活用の切り替え

次に、探索率を変えたときの行動選択を見ます。探索不足は局所最適に閉じる典型的な原因です。

ここで注目してほしいのは、処理の入出力がどの変数で接続されるかです。コードを読むときは『この行は何を決める行か』を短くメモしながら進むと理解が安定します。

In [ ]:
def choose_action(q_left, q_right, epsilon):
    if epsilon > 0.3:
        return 'explore'
    return 'left' if q_left >= q_right else 'right'
print(choose_action(0.4, 0.7, 0.5), choose_action(0.4, 0.7, 0.1))

探索率は固定せず、学習段階に応じて減衰させるのが一般的です。初期は広く探索し、後半で活用へ寄せます。

たとえば 状態 を別の値に変更すると、出力がどう変わるかを確認してください。次に、同じコードで境界条件を試し、なぜその結果になるのかを言葉で説明します。

この確認を繰り返すと、理論と実装の対応が頭の中で一本化されます。

## 5. 方策評価の簡易チェック

最後に、方策の平均報酬を簡易的に比較します。アルゴリズムの評価は、更新式だけでなく結果の検証が不可欠です。

ここで注目してほしいのは、処理の入出力がどの変数で接続されるかです。コードを読むときは『この行は何を決める行か』を短くメモしながら進むと理解が安定します。

In [ ]:
episode_rewards = [1.2, 0.8, 1.5, 1.1, 1.4]
avg_reward = sum(episode_rewards) / len(episode_rewards)
variance = sum((r - avg_reward) ** 2 for r in episode_rewards) / len(episode_rewards)
print('avg =', round(avg_reward, 4))
print('var =', round(variance, 4))

平均だけでなく分散を見ると、方策の安定性も評価できます。実運用ではこの二軸が重要です。

たとえば 状態 を別の値に変更すると、出力がどう変わるかを確認してください。次に、同じコードで境界条件を試し、なぜその結果になるのかを言葉で説明します。

この確認を繰り返すと、理論と実装の対応が頭の中で一本化されます。

## つまずきポイントと確認課題

実装学習では、書けることより説明できることを重視します。次の失敗パターンを先に知っておくと、学習速度が上がります。

1. 探索率が低すぎて行動が固定化する
2. 報酬設計が目的とずれている
3. 長期ロールアウトの不安定性を検証しない

思考実験: 即時報酬が低い行動が、将来報酬で逆転する状況を自作します。

確認課題
1. このノートのコードのうち1つを選び、行ごとの役割を口頭で説明する。
2. 定数を1つ変更し、出力変化の理由を『入力 -> 中間値 -> 出力』で説明する。
3. 実務利用を想定し、異常入力時に壊れる箇所と防止策を1つ提案する。

次の「価値反復法」では、同じ確認手順を維持したまま対象を広げます。今回のコードを再実行してから進むと理解がつながります。

補講 1

強化学習 の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 2

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 3

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 4

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 5

強化学習 の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 6

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 7

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 8

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 9

強化学習 の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 10

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 11

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 12

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 13

強化学習 の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 14

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 15

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 16

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 1

強化学習 の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 2

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 3

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 4

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 5

強化学習 の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 6

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 7

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 8

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 9

強化学習 の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 10

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 11

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 12

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 13

強化学習 の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 14

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 15

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 16

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 1

強化学習 の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 2

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 3

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 4

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 5

強化学習 の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。

補講 6

デバッグ時の基本姿勢は、入力・中間値・出力の三点を固定して比較することです。うまく動かないとき、いきなり大きな修正を入れると原因を見失います。まずは1つの定数だけを変更し、どの中間値が変わったかを観察してください。この小さな観察を積み重ねることが、実装力を最短で伸ばす道です。

補講 7

初学者がつまずく典型は、正しいコードと正しい説明を別々に持ってしまうことです。コードは動くが言葉で説明できない、あるいは説明はできるがコードに落とせない、という分離が起きます。これを防ぐには、実行後に『この行は何を更新したか』を一文で言う練習が有効です。

補講 8

評価設計の観点でも、実装の意味を言葉で固定することが重要です。単一の数値だけを見ると、改善の方向を誤ることがあります。できるだけ複数の観点、例えば平均誤差と外れ値、短期挙動と長期挙動を同時に観察し、どの観点で改善したかを明示してください。

補講 9

強化学習 の学習で最も効果があるのは、コードを読んだあとに式へ戻る往復です。先に式だけを追うと抽象度が高く、初学者は何を計算しているかを見失いやすくなります。逆に、先にコードを動かして値の流れを掴んでから式へ戻ると、記号の役割が具体的に見えてきます。